<h1 style="text-align: center;"><center>Master&nbsp;in&nbsp;Interdisciplinary&nbsp;and&nbsp;Innovation&nbsp;Engineering Computer&nbsp;Vision</center></h1>
<h2 style="text-align: center;"><center><strong>Convolutional Neural Networks - Practice 4</strong></center></h2>
<hr />
<p style="text-align: center;"><center><br />Departament de Matem&agrave;tiques (DMAT)<br />Escola d&rsquo;Enginyeria de Barcelona Est (EEBE)<br />Universitat Polit&egrave;cnica de Catalunya (UPC)<br />2023</center></p>
<ul>
<li>CNN Examples</li>
<li>Quantitative metrics</li>
</ul>


---




<h1>Practice 4</h1>

The **Micro Organism dataset** is a collection of images of different types of microorganisms, with the goal of facilitating the classification of these organisms using machine learning algorithms. The dataset contains 8 different folders, each representing a specific type of microorganism, along with the number of images contained in each folder:

- **Amoeba**: 72 images
- **Euglena**: 168 images
- **Hydra**: 76 images
- **Paramecium**: 152 images
- **Rod bacteria**: 85 images
- **Spherical bacteria**: 86 images
- **Spiral bacteria**: 75 images
- **Yeast**: 75 images

The images in the dataset have varying sizes and resolutions, and may require preprocessing before being used for training deep learning models. The dataset provides a valuable resource for researchers and developers working on image classification and recognition tasks in the field of microbiology.

Link: https://www.kaggle.com/datasets/mdwaquarazam/microorganism-image-classification

---
# 1. Data set
---

- **os:** It is a Python library for interacting with the operating system, it provides a portable way to use operating system dependent functionality.

- **pandas:** It is a library for data manipulation and analysis. It provides data structures for efficiently storing and manipulating large datasets.

- **sklearn:** It is a machine learning library for Python. It provides tools for data preprocessing, model selection, and evaluation, as well as a range of machine learning algorithms.

- **train_test_split:** It is a function from the sklearn library that splits a dataset into training and testing sets. This is commonly used to evaluate the performance of machine learning models.

- **pathlib:** It is a library for working with file paths. 

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**1.1 Path** is used to access training and validation data that is stored in a specific location.

In [ ]:
# Location of the database on Google Drive
path = Path('/content/drive/MyDrive/I&I MASTER /Computer Vision/Data/Micro_Organism')

# Create a list with the names of all the folders in the main folder
folders = os.listdir(path)

In [ ]:
# Create an empty list to store the information of each image
data_f = []

# Iterate over each folder and add the information of each image to the data list
for folder in folders:
    folder_path = os.path.join(path, folder)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        data_f.append((img_path, folder))


**1.2 Dataframe** is a two-dimensional data structure used to store and manipulate data in Python. We use a dataframe to organize and store the address of each image and the corresponding label.

In [ ]:
# Convert the data list to a Pandas dataframe
df = pd.DataFrame(data_f, columns = ['image_path', 'label'])

**1.3 Split the dataset** on training, validating, and testing sets is a commonly used technique in machine learning to assess the classificability of a machine learning model and to prevent overfitting.

*   The **training set** is used to fit the model parameters.
*   The **validation set** is used to fit the hyperparameters of the model
*   The **test set** is used to evaluate the final performance of the model.



In [ ]:
# Split 80/10/10
train_df, test_df = train_test_split(df, test_size = 0.2, random_state = 42, stratify = df['label'])
val_df, test_df = train_test_split(test_df, test_size = 0.5, random_state = 42, stratify = test_df['label'])

In [ ]:
# Print the size of each data set
print(f"Training set: {len(train_df)}")
print(f"Validation set: {len(val_df)}")
print(f"Test set: {len(test_df)}")

Training set: 631
Validation set: 79
Test set: 79



*   We create a dataframe that is the union of tran_df, val_df and test_df. The difference is that it has a new column **set** where it indicates to which set each image belongs.it belongs.

In [ ]:
train_df['set'] = 'train_set'
val_df['set'] = 'val_set'
test_df['set'] = 'test_set'

df = pd.concat([train_df, val_df, test_df], ignore_index = True, sort = False)

In [ ]:
df.head()

,image_path,label,set
0,/content/drive/MyDrive/I&I MASTER /Computer Vision/Data/Micro_Organism/Spherical_bacteria/Image_43.jpg,Spherical_bacteria,train_set
1,/content/drive/MyDrive/I&I MASTER /Computer Vision/Data/Micro_Organism/Hydra/Image_11.jpg,Hydra,train_set
2,/content/drive/MyDrive/I&I MASTER /Computer Vision/Data/Micro_Organism/Euglena/Image_156.jpg,Euglena,train_set
3,/content/drive/MyDrive/I&I MASTER /Computer Vision/Data/Micro_Organism/Rod_bacteria/Image_17.jpg,Rod_bacteria,train_set
4,/content/drive/MyDrive/I&I MASTER /Computer Vision/Data/Micro_Organism/Amoeba/Image_28.jpg,Amoeba,train_set


---
# 2. Load and process the data
---

**2.1 Inputs**

- **fastai:** It is a library for deep learning and machine learning, built on top of PyTorch. 

- **vision:** It is a module within the fastai library that provides tools for computer vision. This includes pre-processing and data augmentation functions, as well as pre-built models for image classification, object detection, and more.

- **all:** It is a sub-module within the vision module that provides a convenient way to import all of the functions and classes in the vision module in a single line of code. 

In [ ]:
from fastai.vision.all import *

- **batch_size:** Is the size of the batch used to load the data into the model.
- **train_idx and val_idx:** These are index lists that specify the indices of the images to be used for training and validation.

In [ ]:
batch_size = 128
train_idx = list(train_df.index)
val_idx = list(val_df.index)

**2.2 Process the data**
- **ImageDataLoaders:** Is a class in the fastai vision module that is used to load image data and create PyTorch DataLoader objects.
- **cols:** Is a list of columns in the DataFrame that specify the path to each image.
- **Resize:** Is a transform that resizes images to a specific size.
- **aug_transforms:** Is a function that returns a list of random transformations to augment the training data.
- **Normalize.from_stats:** Is a function that normalizes images using the statistics from the ImageNet database.

In [ ]:
data = ImageDataLoaders.from_df(df,
                                path = '/',
                                cols = 'image_path',
                                item_tfms=Resize(224),
                                batch_tfms = [*aug_transforms(), Normalize.from_stats(*imagenet_stats)],
                                bs = batch_size,
                                train_idx = train_idx,
                                valid_idx = val_idx)

In [ ]:
data.show_batch() # show the dataset in random batch

**2.2 Outputs**

In [ ]:
df.label.value_counts()

---
# 3. CNN structure
---

**3.1 Define the CNN architecture**
*   **Models** provides pre-trained fastai neural network architectures.

In [ ]:
from fastai.# student work
print(# student work

- **cnn_learner:** It is a function used for creating a convolutional neural network (CNN). 

- **data:** It is a variable that represents the data used for training the CNN model.

- **resnet18:** It is a pre-trained CNN model architecture available in Fastai. It has 18 layers and is widely used in classification tasks.

- **metrics:** It is a parameter of the cnn_learner function that specifies the evaluation metric to be used during training and validation.

- **accuracy:** It is a metric used to evaluate the performance of a classification model. It measures the percentage of correctly classified samples out of all the samples.

In [ ]:
learn = cnn_learner(# student work

---
# 4. Train the CNN
---

- **fit():** It is a method used to train a deep learning model from 0 on a dataset. During training, the model's weights are adjusted to minimize a loss function. 

- **fine_tune():** It is a method used for fine-tuning a pre-trained convolutional neural network (CNN) model. Fine-tuning involves training the model on a new dataset, usually with a smaller learning rate than the initial pre-training, and updating the weights of the model based on the new data. This method automates the process of fine-tuning and makes it easy to train a model on new data without starting from scratch.




In [ ]:
#learn.fit(20, lr=1e-3)
#learn.fine_tune(20)

---
# 5. Evaluate the CNN on the test set
---
**5.1 Test set** in the trained CNN
- **test_dl:** It is a DataLoader created from the test data, is used to create batches of test data for evaluation. The with_labels=True parameter indicates that the test data has labels.

- **ClassificationInterpretation:** is used for interpreting the results of a classification model. 

- **from_learner:**  method is used to create an instance of the class from the trained CNN model

In [ ]:
test_dl = # student work
interp = # student work

**5.2 Confusion Matrix** 

In [ ]:
interp.plot_# student work

**5.3 Quantitative metrics** 
- **print_classification_report()** prints a report of various classification metrics for the model's predictions on the validation set. Specifically, it prints a table that includes the precision, recall, f1-score, and support for each class in the set, as well as the accuracy.

---

**Precision**: 

$$Precision = \frac{TP}{TP + FP}$$

where TP stands for true positives and FP stands for false positives.

---

**Recall:**

$$Recall = \frac{TP}{TP + FN}$$

where TP stands for true positives and FN stands for false negatives.

---

**F1-score:**

$$F1 = 2 \cdot \frac{Precision \cdot Recall}{Precision + Recall}$$

where Precision and Recall are the metrics previously defined.

---

**Support** refers to the number of samples in the set that belong to each class.

---

**Accuracy** represents the overall fraction of correctly classified samples:

$$Accuracy = \frac{TP + TN}{TP + TN + FP + FN}$$

where TP stands for true positives, TN stands for true negatives, FP stands for false positives and FN stands for false negatives.

---

In [ ]:
interp.print_classification_# student workb

**5.4 Best and worse ranked by CNN** 

*   **Best**



In [ ]:
interp.plot_top_losses(# student work

*   **Worse**

In [ ]:
interp.plot_top_losses(# student work

---
# 6. Covid X-Ray Dataset
---

The Covid_X-Ray_Dataset is a collection of X-ray images of COVID-19 infected patients and non-COVID normal cases. The dataset contains two subdirectories:

- **COVID**: 400 X-ray images of COVID-19 infected patients
- **Normal**: 400 X-ray images of non-COVID cases

This dataset provides a valuable resource for researchers and developers working on image classification and recognition tasks in the field of medical imaging.

Link: https://www.kaggle.com/datasets/mrtejas/covid-19-and-normal-x-ray-dataset-balanced?resource=download

In [ ]:
path = Path('/content/drive/MyDrive/datasets/Covid_X_ray')










# student work










data.show_batch()

In [ ]:
# 
learn = cnn_learner(# student work
learn.fine_tune(20)

In [ ]:
# 
test_dl = # student work
interp = # student work
interp.plot_confusion_# student work
interp.print_# student work
interp.plot_top_losses(# student work
interp.plot_top_losses(# student work

---
# 7. Alzheimer MRI Preprocessed Dataset
---

The Alzheimer MRI Preprocessed Dataset is a collection of 6400 preprocessed MRI images, resized to 128x128 pixels. The dataset consists of four different classes of images, each representing a different level of dementia, with the following number of images in each class:

- Class 1: **Mild Demented** (896 images)
- Class 2: **Moderate Demented** (64 images)
- Class 3: **Non Demented** (3200 images)
- Class 4: **Very Mild Demented** (2240 images)

The data was collected from several websites, hospitals, and public repositories. The dataset provides a valuable resource for researchers and developers working on image classification and recognition tasks in the field of Alzheimer's research.

Link: https://www.kaggle.com/datasets/sachinkumar413/alzheimer-mri-dataset

In [ ]:
path = Path('/content/drive/MyDrive/datasets/Alzheimer')









# student work









data.show_batch()

In [ ]:
# 
learn = cnn_learner(# student work# student work
learn.fine_tune(20)

In [ ]:
# 
test_dl = # student work
interp = # student work
interp.plot_# student work
interp.print_# student work
interp.plot_top_losses(# student work
interp.plot_top_losses(# student work